# SwissVotes - Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

## Data cleaning

In [ ]:
df_full = pd.read_csv('swiss-votes.csv', delimiter=';')
df_full.head()

In [ ]:
df = pd.DataFrame()
df['title_fr'] = df_full['titel_off_f'] # Title of the vote in French
df['date'] = pd.to_datetime(df_full['datum'], dayfirst=True)
df['date_year'] = df['date'].dt.year
df['type'] = df_full['rechtsform'].map({
    1: 'mandatory-referendum',
    2: 'optional-referendum',
    3: 'popular-initiative',
    4: 'AA',
    5: 'BB'
})
df['author'] = df_full['urheber']
df['valid_signatures'] = df_full['unter_g']
df

### Political parties

In [ ]:
def has_svp(authors): # UDC: Union Democrate du Centre
    authors = authors.lower()
    return "schweizerische volkspartei" in authors or re.search(r'\bsvp\b', authors) is not None

def has_sp(authors): # PS: Parti Socialiste
    authors = authors.lower()
    return "sozialdemokratischen partei" in authors \
        or "sozialdemokratische partei" in authors \
        or re.search(r'\bsp\b', authors) is not None

def has_mitte(authors): # Partis centristes
    authors = authors.lower()
    return re.search(r'\b(cvp|bdp)\b', authors) is not None

def has_fdp(authors): # Parti libéral-démocrate
    authors = authors.lower()
    return re.search(r'\bfdp\b', authors) is not None

def has_grun(authors): # Partis verts
    authors = authors.lower().replace('ü', 'u')
    return re.search(r'\bgrun(e|en|es)?\b', authors) is not None

df['author_partei_svp'] = df['author'].map(has_svp)
df['author_partei_sp'] = df['author'].map(has_sp)
df['author_partei_mitte'] = df['author'].map(has_mitte)
df['author_partei_fdp'] = df['author'].map(has_fdp)
df['author_partei_grun'] = df['author'].map(has_grun)

df['author_partei'] = df['author_partei_svp'] | df['author_partei_sp'] | df['author_partei_mitte'] | df['author_partei_fdp'] | df['author_partei_grun']

POLITICAL_PARTIES = {
    'svp': "Swiss People's Party",
    'sp': "Social Democratic Party",
    #'mitte': "The Centre",
    #'fdp': "Free Democratic Party",
    'grun': "Green parties",
}

No author is provided for mandatory referendums (since the referendum is not initiated by a specific group of people). A negligible number of votes of another type also miss this information.

In [ ]:
df.query("type != 'mandatory-referendum' & author == '.'")

In [ ]:
df_initiatives = df.query("type == 'popular-initiative'")

print('Percentage of popular initiatives from a political party', df_initiatives['author_partei'].sum() / len(df_initiatives))

In [ ]:
print("Number of initiatives per party:")
print(sum(df['author_partei_svp']), sum(df['author_partei_sp']), sum(df['author_partei_mitte']), sum(df['author_partei_fdp']), sum(df['author_partei_grun']))

In [ ]:
df[df['author_partei_grun']]['author']

In [ ]:
for _, row in df.iterrows():
    aut = row['author']
    if 'partei' in aut.lower() and not row['author_partei']:
        print(aut)

In [ ]:
for _, row in df.iterrows():
    aut = row['author']
    if 'grun' in aut.lower().replace('ü', 'u') and not has_grun(aut):
        print(aut)

In [ ]:
from collections import Counter
from matplotlib_venn import venn3

counter = Counter()

for _, row in df.iterrows():
    votes = (row['author_partei_sp'], row['author_partei_grun'], row['author_partei_svp'])

    if votes != (0, 0, 0):
        counter.update([votes])

venn3(
    subsets=(counter[(True, False, False)], counter[(False, True, False)], counter[(True, True, False)], counter[(False, False, True)], counter[(True, False, True)], counter[(False, True, True)], counter[(True, True, True)]),
    set_labels=(POLITICAL_PARTIES['sp'], POLITICAL_PARTIES['grun'], POLITICAL_PARTIES['svp'])
)
plt.show()

In [ ]:
for id in POLITICAL_PARTIES:
    df_party = df.query(f"author_partei_{id} == 1")
    plt.figure(figsize=(10, 6))
    sns.histplot(df_party['date_year'], bins=(df_party['date_year'].max()-df_party['date_year'].min()) // 10)
    plt.title(f"Number of popular votes per year for {POLITICAL_PARTIES[id]}")
    plt.xlabel('Year')
    plt.ylabel('Number of votes')
    plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

for id in ['sp', 'svp', 'grun']:
    df_party = df.query(f"author_partei_{id} == 1")

    #sns.histplot(df_party['date_year'], bins=(df_party['date_year'].max()-df_party['date_year'].min()) // 10)
    sns.histplot(df_party['date_year'], binwidth=5, multiple="dodge")

plt.title(f"Number of popular initiatives per year by party")
plt.xlabel('Year')
plt.ylabel('Number of votes')
plt.show()

## Analysis

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['date_year'], binwidth=10)
plt.title('Number of popular votes per decade')
plt.xlabel('Year')
plt.ylabel('Number of votes')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_initiatives['date_year'], binwidth=10)
plt.title('Number of popular initiatives per decade')
plt.xlabel('Year')
plt.ylabel('Number of votes')
plt.show()

In [ ]:
sns.displot(df, x='date_year', hue='type', kind='hist', multiple='fill', aspect=2, height=6)
plt.title("Proportion of the referendum used over the years")
plt.xlabel("Year")
plt.xlim(df['date_year'].min(), df['date_year'].max())
plt.ylabel("Proportion")
plt.show()